In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('dates').getOrCreate()

In [3]:
%fs ls /FileStore/tables

path,name,size
dbfs:/FileStore/tables/ContainsNull.csv,ContainsNull.csv,61
dbfs:/FileStore/tables/Spark_Essentials-5d27c.dbc,Spark_Essentials-5d27c.dbc,1414841
dbfs:/FileStore/tables/appl_stock.csv,appl_stock.csv,143130
dbfs:/FileStore/tables/cogsley_clients.csv,cogsley_clients.csv,384219
dbfs:/FileStore/tables/cogsley_sales.csv,cogsley_sales.csv,2176442
dbfs:/FileStore/tables/people.json,people.json,73
dbfs:/FileStore/tables/sales_info.csv,sales_info.csv,196
dbfs:/FileStore/tables/state_info.csv,state_info.csv,2778


In [4]:
path = '/FileStore/tables/appl_stock.csv'

In [5]:
df = spark.read.csv(path,inferSchema=True,header=True)

In [6]:
df.show(5)

+-------------------+----------+----------+------------------+------------------+---------+------------------+
 Date| Open| High| Low| Close| Volume| Adj Close|
+-------------------+----------+----------+------------------+------------------+---------+------------------+
2010-01-04 00:00:00|213.429998|214.499996|212.38000099999996| 214.009998|123432400| 27.727039|
2010-01-05 00:00:00|214.599998|215.589994| 213.249994| 214.379993|150476200|27.774976000000002|
2010-01-06 00:00:00|214.379993| 215.23| 210.750004| 210.969995|138040000|27.333178000000004|
2010-01-07 00:00:00| 211.75|212.000006| 209.050005| 210.58|119282800| 27.28265|
2010-01-08 00:00:00|210.299994|212.000006|209.06000500000002|211.98000499999998|111902700| 27.464034|
+-------------------+----------+----------+------------------+------------------+---------+------------------+
only showing top 5 rows

In [7]:
df.printSchema()

root
-- Date: timestamp (nullable = true)
-- Open: double (nullable = true)
-- High: double (nullable = true)
-- Low: double (nullable = true)
-- Close: double (nullable = true)
-- Volume: integer (nullable = true)
-- Adj Close: double (nullable = true)

In [8]:
from pyspark.sql.functions import (dayofmonth,dayofyear,weekofyear, month,
                                   hour,year,format_number,date_format)

In [9]:
df.select(dayofmonth(df['Date'])).show()

+----------------+
dayofmonth(Date)|
+----------------+
 4|
 5|
 6|
 7|
 8|
 11|
 12|
 13|
 14|
 15|
 19|
 20|
 21|
 22|
 25|
 26|
 27|
 28|
 29|
 1|
+----------------+
only showing top 20 rows

In [10]:
df.select(weekofyear(df['Date'])).show()

+----------------+
weekofyear(Date)|
+----------------+
 1|
 1|
 1|
 1|
 1|
 2|
 2|
 2|
 2|
 2|
 3|
 3|
 3|
 3|
 4|
 4|
 4|
 4|
 4|
 5|
+----------------+
only showing top 20 rows

In [11]:
new_df = df.withColumn('Year',year(df['Date']))

In [12]:
new_df.show(5)

+-------------------+----------+----------+------------------+------------------+---------+------------------+----+
 Date| Open| High| Low| Close| Volume| Adj Close|Year|
+-------------------+----------+----------+------------------+------------------+---------+------------------+----+
2010-01-04 00:00:00|213.429998|214.499996|212.38000099999996| 214.009998|123432400| 27.727039|2010|
2010-01-05 00:00:00|214.599998|215.589994| 213.249994| 214.379993|150476200|27.774976000000002|2010|
2010-01-06 00:00:00|214.379993| 215.23| 210.750004| 210.969995|138040000|27.333178000000004|2010|
2010-01-07 00:00:00| 211.75|212.000006| 209.050005| 210.58|119282800| 27.28265|2010|
2010-01-08 00:00:00|210.299994|212.000006|209.06000500000002|211.98000499999998|111902700| 27.464034|2010|
+-------------------+----------+----------+------------------+------------------+---------+------------------+----+
only showing top 5 rows

In [13]:
new_df.groupBy('Year').mean().show()

+----+------------------+------------------+------------------+------------------+--------------------+------------------+---------+
Year| avg(Open)| avg(High)| avg(Low)| avg(Close)| avg(Volume)| avg(Adj Close)|avg(Year)|
+----+------------------+------------------+------------------+------------------+--------------------+------------------+---------+
2015|120.17575393253965|121.24452385714291| 118.8630954325397|120.03999980555547| 5.18378869047619E7|115.96740080555561| 2015.0|
2013| 473.1281355634922| 477.6389272301587|468.24710264682557| 472.6348802857143| 1.016087E8| 62.61798788492063| 2013.0|
2014| 295.1426195357143|297.56103184523823| 292.9949599801587| 295.4023416507935| 6.315273055555555E7| 87.63583323809523| 2014.0|
2012| 576.652720788| 581.8254008040001| 569.9211606079999| 576.0497195640002| 1.319642044E8| 74.81383696800002| 2012.0|
2016|104.50777772619044| 105.4271825436508|103.69027771825397|104.60400786904763| 3.84153623015873E7|103.15032854761901| 2016.0|
2010| 259.9576190992064|262.36880881349214|256.84761791269847| 259.8424600000002|1.4982631666666666E8|33.665072424603196| 2010.0|
2011|364.06142773412705| 367.4235704880951|360.29769878174613|364.00432532142867|1.2307474166666667E8| 47.16023692063492| 2011.0|
+----+------------------+------------------+------------------+------------------+--------------------+------------------+---------+

In [14]:
result = new_df.groupBy('Year').mean().select(['Year','avg(Close)'])

In [15]:
result.show()

+----+------------------+
Year| avg(Close)|
+----+------------------+
2015|120.03999980555547|
2013| 472.6348802857143|
2014| 295.4023416507935|
2012| 576.0497195640002|
2016|104.60400786904763|
2010| 259.8424600000002|
2011|364.00432532142867|
+----+------------------+

In [16]:
results = result.withColumnRenamed('avg(Close)','Average Closing Price')

In [17]:
final_results =results.select(['Year',format_number('Average Closing Price',2).alias('Average closing price')]).show()

+----+---------------------+
Year|Average closing price|
+----+---------------------+
2015| 120.04|
2013| 472.63|
2014| 295.40|
2012| 576.05|
2016| 104.60|
2010| 259.84|
2011| 364.00|
+----+---------------------+